<a href="https://colab.research.google.com/github/SharatGupta/DSML-Projects/blob/main/DAV%20-%20Propability%20%26%20Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [ ]:
def normal(mu, sigma,obs):
  # Sigma is the Population standard Deviation
  # mu is Pupulation or Sample Mean
  # obs is value for which propability to be calculated
  z = (obs- mu)/sigma
  prob_less = norm.cdf(z)
  prob_more = 1 - prob_less
  return prob_less, prob_more

In [ ]:
def clt_fun(mu, sigma, n, obs):
  # Sigma is the Population standard Deviation
  # mu is Pupulation or Sample Mean
  # n is sample Size
  # obs is value for which propability to be calculated
  std_error = sigma / np.sqrt(n)
  z = (obs- mu)/std_error
  prob_less = norm.cdf(z)
  prob_more = 1 - prob_less
  return prob_less, prob_more

In [ ]:
clt_fun(80, 15, 50, 75)

(0.009211062727049501, 0.9907889372729505)

In [ ]:
def confiinterval(mu, sigma, n, perct):
  # Sigma is the Population standard Deviation
  # mu is Pupulation or Sample Mean
  # n is sample Size
  # perct is percentatge for interval calculation
  std_error = sigma / np.sqrt(n)
  x1 = mu + (std_error * norm.ppf((100-perct)/200))
  x2 = mu + (std_error * norm.ppf(1 - ((100-perct)/200)))
  return x1,x2